In [2]:
import datetime
import pandas as pd
from selenium import webdriver
import re
import time
import numpy as np

In [3]:
def actualise_df(df):
    browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div[1]/div/div/div[1]/div/i[1]').click()
    time.sleep(3)
    matches = browser.find_element_by_class_name('stats-section-title__games-count').text
    matches = int(re.findall('\d+', matches)[0])
    games = []
    for i in range(3, matches + 3):
        game = browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div[2]/div[1]/div/div/div[{}]/div[2]/div[1]/div/div[1]'.format(i))
        games.append(game.text)
    lst_infos = []
    for game in games:
        infos = []
        teams = re.findall(r'[A-Z]{3}', game)
        scores = re.findall(r'(?<=\d\s)\d+', game)
        bilans = re.findall(r'\d+-\d+', game)
        infos.extend([teams[1], teams[0]])
        over_time_periods = int((len(scores) - 10) / 2)
        if over_time_periods > 0:
            over_time_scores_dom = []
            over_time_scores_ext = []
            for i in range(0, over_time_periods):
                over_time_scores_ext.append(scores.pop(4))
            for i in range(0, over_time_periods):
                over_time_scores_dom.append(scores.pop(9))
            over_time_scores_dom = list(map(int, over_time_scores_dom))
            over_time_scores_ext = list(map(int, over_time_scores_ext))
            over_time_scores_dom = sum(over_time_scores_dom)
            over_time_scores_ext = sum(over_time_scores_ext)
        for i in range(0,5):
            infos.append('{}-{}'.format(scores[i + 5], scores[i]))
        infos.append(over_time_periods)
        if over_time_periods > 0:
            infos.append('{}-{}'.format(over_time_scores_dom, over_time_scores_ext))
        else:
            infos.append(np.nan)
        infos.extend([bilans[1], bilans[0]])
        date = browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div[1]/div/div/div[1]/div/div/a/span[1]').text
        infos.append(date)
        lst_infos.append(infos)
    df_len = len(df) + 1
    for i in range(0, matches):
        df.loc[df_len + i] = lst_infos[i]
    return(df) 

In [ ]:
def get_match_stats():
     

In [89]:
#récuperer le dataframe
try:
    df = pd.read_pickle("./nba_results.pkl")
except FileNotFoundError:
    df = pd.DataFrame(columns=['Domicile', 'Exterieur', 'Q1', 'Q2', 'Q3', 'Q4', 'Final', 'Nb_OverTime', 'OverTime_Score', 'Bilan_dom', 'Bilan_ext', 'Date'])
df

,Domicile,Exterieur,Q1,Q2,Q3,Q4,Final,Nb_OverTime,OverTime_Score,Bilan_dom,Bilan_ext,Date


In [4]:
#LANCER LE NAVIGUATEUR
now = datetime.datetime.now()
now = "{:02d}/{:02d}/{:4d}".format(now.month, now.day, now.year)
browser = webdriver.Firefox(executable_path='../Documents/geckodriver.exe')
browser.get('https://stats.nba.com/scores/{}'.format(now))

In [95]:
for i in range(0, 20):
    df = actualise_df(df)
df[df['Nb_OverTime'] != 0]

['14']
['39', '25', '22', '34', '134', '26', '20', '36', '38', '18', '138']
['18']
['39', '25', '22', '34', '134', '26', '20', '36', '38', '138']
['16']
['27', '40', '28', '27', '138', '39', '24', '33', '26', '6', '128']
['6']
['27', '40', '28', '27', '138', '39', '24', '33', '26', '128']
['6']
['34', '29', '21', '25', '115', '26', '32', '32', '19', '11', '120']
['11']
['34', '29', '21', '25', '115', '26', '32', '32', '19', '120']
['14']
['34', '28', '23', '46', '145', '30', '24', '40', '37', '11', '142']
['11']
['34', '28', '23', '46', '145', '30', '24', '40', '37', '142']
['7']
['36', '25', '37', '26', '9', '140', '21', '29', '34', '40', '7', '7', '138']
['7', '9']
['36', '25', '37', '26', '140', '21', '29', '34', '40', '7', '7', '138']
['7']
['36', '25', '37', '26', '140', '21', '29', '34', '40', '7', '138']
['7', '7']
['36', '25', '37', '26', '140', '21', '29', '34', '40', '138']
['11']
['23', '40', '30', '37', '6', '147', '33', '37', '35', '25', '11', '13', '154']
['11', '6']
['23

,Domicile,Exterieur,Q1,Q2,Q3,Q4,Final,Nb_OverTime,OverTime_Score,Bilan_dom,Bilan_ext,Date
1,CLE,BKN,29-30,29-25,26-29,25-25,139-148,3,30-39,12-46,30-29,"WEDNESDAY, FEB 13"
100,LAC,LAL,26-32,27-22,22-33,37-25,120-123,1,8-11,28-24,27-25,"THURSDAY, JAN 31"
105,MIN,MEM,24-16,20-28,27-25,22-24,99-97,1,6-4,25-26,20-32,"WEDNESDAY, JAN 30"
235,HOU,LAL,26-39,20-25,36-22,38-34,138-134,1,18-14,26-19,25-22,"SATURDAY, JAN 19"
249,OKC,LAL,39-27,24-40,33-28,26-27,128-138,1,6-16,26-18,25-21,"THURSDAY, JAN 17"
250,DET,ORL,26-34,32-29,32-21,19-25,120-115,1,11-6,19-24,19-25,"WEDNESDAY, JAN 16"
252,HOU,BKN,30-34,24-28,40-23,37-46,142-145,1,11-14,25-19,23-23,"WEDNESDAY, JAN 16"
271,WAS,TOR,21-36,29-25,34-37,40-26,138-140,2,14-16,18-26,33-12,"SUNDAY, JAN 13"
296,SAS,OKC,33-23,37-40,35-30,25-37,154-147,2,24-17,25-18,25-16,"THURSDAY, JAN 10"


,Domicile,Exterieur,Q1,Q2,Q3,Q4,Nb OverTime,OverTime Score,Final,Bilan_dom,Bilan_ext,Date
1,CLE,BKN,29-30,29-25,26-29,25-25,139-148,3,12-12,12-46,30-29,"WEDNESDAY, FEB 13"
2,IND,MIL,18-26,29-29,32-21,18-30,97-106,3,12-12,38-20,43-14,"WEDNESDAY, FEB 13"
3,BOS,DET,28-29,29-23,34-20,27-38,118-110,3,12-12,37-21,26-30,"WEDNESDAY, FEB 13"
4,NYK,PHI,33-37,22-34,36-29,20-26,111-126,3,12-12,10-47,37-21,"WEDNESDAY, FEB 13"
5,TOR,WAS,30-26,29-39,44-28,26-27,129-120,3,12-12,43-16,24-34,"WEDNESDAY, FEB 13"
6,CHI,MEM,36-32,31-30,25-21,30-27,122-110,3,12-12,14-44,23-36,"WEDNESDAY, FEB 13"
7,MIN,HOU,28-29,24-29,35-26,34-27,121-111,3,12-12,27-30,33-24,"WEDNESDAY, FEB 13"
8,DAL,MIA,25-29,33-28,16-32,27-23,101-112,3,12-12,26-31,26-30,"WEDNESDAY, FEB 13"
9,DEN,SAC,30-41,28-28,35-22,27-27,120-118,3,12-12,39-18,30-27,"WEDNESDAY, FEB 13"
10,POR,GSW,31-33,33-28,30-34,35-12,129-107,3,12-12,34-23,41-16,"WEDNESDAY, FEB 13"


In [14]:
df.to_pickle('nba_results.pkl')

In [53]:
def actualise_df(df):
    browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div[1]/div/div/div[1]/div/i[1]').click()
    time.sleep(3)
    matches = browser.find_element_by_class_name('stats-section-title__games-count').text
    matches = int(re.findall('\d+', matches)[0])
    games = []
    for i in range(3, matches + 3):
        game = browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div[2]/div[1]/div/div/div[{}]/div[2]/div[1]/div/div[1]'.format(i))
        games.append(game.text)
    lst_infos = []
    for game in games:
        infos = []
        teams = re.findall(r'[A-Z]{3}', game)
        scores = re.findall(r'(?<=\d\s)\d+', game)
        bilans = re.findall(r'\d+-\d+', game)
        infos.extend([teams[1], teams[0]])
        for i in range(0,5):
            infos.append('{}-{}'.format(scores[i + 5], scores[i]))
        infos.extend([bilans[1], bilans[0]])
        date = browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div[1]/div/div/div[1]/div/div/a/span[1]').text
        infos.append(date)
        lst_infos.append(infos)
    df_len = len(df) + 1
    for i in range(0, matches):
        df.loc[df_len + i] = lst_infos[i]
    return(df)

In [39]:
#gerer les prolongations
#ajouter une colonne nombres   prolongation et une avec le score des prolongations
p_game = browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div[2]/div[1]/div/div/div[3]/div[2]/div[1]/div/div[1]').text
print(p_game)
scores = re.findall(r'(?<=\d\s)\d+', p_game)
print(scores)
print(len(scores))
print(scores[0:(len(scores)/2)])
"""np_game = browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div[2]/div[1]/div/div/div[4]/div[2]/div[1]/div/div[1]').text
print(np_game)
scores = re.findall(r'(?<=\d\s)\d+', np_game)
print(scores)
print(len(scores))"""

Q1 Q2 Q3 Q4 OT1 OT2 OT3
BKN 30-29 30 25 29 25 9 10 20 148
CLE 12-46 29 29 26 25 9 10 11 139
['30', '25', '29', '25', '9', '10', '20', '148', '29', '29', '26', '25', '9', '10', '11', '139']
16


TypeError: slice indices must be integers or None or have an __index__ method

In [22]:
df

,Domicile,Exterieur,Q1,Q2,Q3,Q4,Final,Bilan_dom,Bilan_ext,Date
10,DET,NYK,41-22,22-32,26-30,31-19,120-103,25-29,10-44,"FRIDAY, FEB 8"
11,PHI,DEN,33-23,23-33,27-26,34-28,117-110,35-20,37-18,"FRIDAY, FEB 8"
12,WAS,CLE,41-27,25-25,26-34,27-20,119-106,23-32,11-44,"FRIDAY, FEB 8"
13,BKN,CHI,27-32,21-23,32-38,26-32,106-125,29-28,13-42,"FRIDAY, FEB 8"
14,DAL,MIL,29-29,22-35,35-30,21-28,107-122,25-29,41-13,"FRIDAY, FEB 8"
15,PHX,GSW,31-26,22-34,32-22,22-35,107-117,11-46,39-15,"FRIDAY, FEB 8"
16,NOP,MIN,22-35,40-27,28-26,32-29,122-117,25-31,25-30,"FRIDAY, FEB 8"
17,SAC,MIA,17-28,33-31,25-24,27-13,102-96,29-26,25-28,"FRIDAY, FEB 8"
18,IND,LAC,36-31,35-22,27-20,18-19,116-92,36-19,30-26,"THURSDAY, FEB 7"
19,ORL,MIN,32-31,39-27,31-27,20-27,122-112,23-32,25-29,"THURSDAY, FEB 7"
